# point_defect_formation_static
- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-10-07

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The __point_defect_formation_static__ calculation evaluates the formation energy for a variety of point defects and evaluates whether the defect configurations remain stable after relaxation. A perfect system is constructed and the cohesive energy is calculated. Then, a point defect is inserted into the system and the defect's formation energy is computed. The configuration of the defect and neighboring atoms of the relaxed system are used to check if the defect strucutre has relaxed to a lower energy configuration.

__Disclaimer__: this method uses pre-defined point defect configurations and relaxes the systems through energy/force minimization. The lowest energy defect configuration may not be one of the pre-defined ones and may require a more thorough (temperature) annealing to obtain. The algorithms used to identify when a defect structure relaxes to another configuration have been tested for simple materials/defects and may not work for all cases.

## Demonstration

Library Imports

In [1]:
#Standard Python imports
import os
import shutil
from copy import deepcopy
from collections import OrderedDict

#http://www.numpy.org/
import numpy as np

#https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

#https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

import iprPy

### 1. Generate Data Files

We have embedded a few files to make the Notebook a self-contained calculation.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

4. __Point Defect Data Model.__ Collects metadata and input parameters associated with adding a particular point defect to a system.

Create working directory

In [2]:
working_dir = 'point_defect_formation_static'

#This prevents recursive directories upon re-running this cell
if os.path.basename(os.getcwd()) == working_dir:
    working_dir = os.getcwd()
else:
    working_dir = os.path.realpath(working_dir)
    
#move to working_dir
try:
    os.chdir(working_dir)
except:
    os.makedirs(working_dir)
    os.chdir(working_dir)

#### 1.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [3]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
f = open('Jelinek_2012_meamf', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")
f.close()

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
f = open('Jelinek_2012_meam.alsimgcufe', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")
f.close()

#### 1.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential. Additional potential data models and their associated artifacts can be found in the ipyrPy/reference-libraries/potentials directory.

In [4]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w')
f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close()    

#### 1.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc A1 crystal prototype. Additional prototype data models can be found in the ipyrPy/reference-libraries/prototypes directory.

In [5]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
f = open('A1--Cu--fcc.json', 'w')
f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")
f.close()

#### 1.4 Point Defect Data Model

This generates point defect data model files associated with various interstitial positions in the fcc A1 crystal prototype. Additional point defect data models can be found in the ipyrPy/reference-libraries/point directory.

In [6]:
#Creates instances of point defect data models for adding point defects to crystal systems.
f = open('100-dumbbell.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "507c77c1-d903-476a-b859-2e8538e8f7af",
            "id": "100-dumbbell"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [0.0, 0.0, 0.0], 
            "db_vect": [0.0, 0.0, 0.3333333333333], 
            "scale": true
        }
    }
}""")
f.close()

f = open('110-dumbbell.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "e774b498-a339-4689-846a-8ac53acf588e",
            "id": "110-dumbbell"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [0.0, 0.0, 0.0], 
            "db_vect": [0.0, 0.25, 0.25], 
            "scale": true
        }
    }
}""")
f.close()

f = open('111-dumbbell.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "97628465-3654-47b8-9176-12c597424067",
            "id": "111-dumbbell"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [0.0, 0.0, 0.0], 
            "db_vect": [0.25, 0.25, 0.25], 
            "scale": true
        }
    }
}""")
f.close()

f = open('octahedral-interstitial.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "3aa3bb62-f109-468b-b651-3590b0f666c5",
            "id": "octahedral-interstitial"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [0.5, 0.5, 0.5], 
            "scale": true
        }
    }
}""")
f.close()

f = open('crowdion-interstitial.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "0b6d419e-4c27-4143-8e87-58824980c4f8",
            "id": "crowdion-interstitial"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [0.25, 0.25, 0.0], 
            "scale": true
        }
    }
}""")
f.close()

f = open('tetrahedral-interstitial.json', 'w')
f.write("""{
    "point-defect-parameters": {
        "point-defect": {
            "key": "610f3964-c647-45ce-a62e-31941ff007b5",
            "id": "tetrahedral-interstitial"
        }, 
        "system-family": "A1--Cu--fcc", 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [0.25, 0.25, 0.25], 
            "scale": true
        }
    }
}""")
f.close()

### 2. LAMMPS Script Template(s)

This generates any LAMMPS imput template files that are used by the calculation functions. The calculation functions fill in variable terms in the templates to create complete LAMMPS simulation scripts.

1. min.template -- Performs an energy minimization on a system and creates a dump file of the final configuration.

#### 2.1 min.template

In [7]:
f = open('min.template', 'w')
f.write("""#LAMMPS input script that performs a simple energy minimization

<atomman_system_info>

<atomman_pair_info>

variable peatom equal pe/atoms
                  
thermo_style custom step lx ly lz pxx pyy pzz pe v_peatom
thermo_modify format float %.13e

dump dumpit all custom <maximum_evaluations> atom.* id type x y z
dump_modify dumpit format "%i %i %.13e %.13e %.13e"
minimize <energy_tolerance> <force_tolerance> <maximum_iterations> <maximum_evaluations>""")
f.close()

### 3. Python Calculation Function(s)

These are the Python functions that are at the heart of performing the calculation.

1. __ptd_energy__: evaluates the energy of a system before and after a point defect has been added to compute the defect formation energy.

2. __check_ptd_config__: looks at the final relaxed point defect system to determine if the defect has relaxed to a different configuration.

#### 3.1 ptd_energy Function

Adds one or more point defects to a system and evaluates the defect formation energy.

Arguments:
    
- __lammps_command__ -- command for running LAMMPS.

- __system__ -- atomman.System to add the point defect to.

- __potential__ -- atomman.lammps.Potential representation of a LAMMPS implemented potential.

- __symbols__ -- list of element-model symbols for the Potential that correspond to system's atypes.

- __ptd_model__ -- DataModelDict representation of a point defect data model.
    
Keyword Arguments:
    
- __mpi_command__ -- MPI command for running LAMMPS in parallel. Default value is None (serial run).  

- __etol__ -- energy tolerance to use for the LAMMPS minimization. Default value is 0.0 (i.e. only uses ftol). 

- __ftol__ -- force tolerance to use for the LAMMPS minimization. Default value is 1e-6.

- __maxiter__ -- the maximum number of iterations for the LAMMPS minimization. Default value is 100000.

- __maxeval__ -- the maximum number of evaluations for the LAMMPS minimization. Default value is 100000.

Returns a dictionary containing:

- __E_coh__ -- the cohesive energy of the defect-free base system.

- __E_ptd_f__ -- the point defect formation energy.

- __E_total_base__ -- the total energy of the defect-free base system.

- __E_total_ptd__ -- the total energy of the defect containing system.

- __system_base__ -- an atomman.System of the relaxed defect-free base system.

- __system_ptd__ -- an atomman.System of the relaxed defect containing system.

This code is directly copied from calc_point_defect_formation_static.py.

In [8]:
def ptd_energy(lammps_command, system, potential, symbols, ptd_model, mpi_command=None,
               etol=0.0, ftol=1e-6, maxiter=100000, maxeval=100000):
    """
    Adds one or more point defects to a system and evaluates the defect formation energy.
    
    Arguments:
    lammps_command -- command for running LAMMPS.
    system -- atomman.System to add the point defect to.
    potential -- atomman.lammps.Potential representation of a LAMMPS implemented potential.
    symbols -- list of element-model symbols for the Potential that correspond to system's atypes.
    ptd_model -- DataModelDict representation of a point defect data model.
    
    Keyword Arguments:
    mpi_command -- MPI command for running LAMMPS in parallel. Default value is None (serial run).  
    etol -- energy tolerance to use for the LAMMPS minimization. Default value is 0.0 (i.e. only uses ftol). 
    ftol -- force tolerance to use for the LAMMPS minimization. Default value is 1e-6.
    maxiter -- the maximum number of iterations for the LAMMPS minimization. Default value is 100000.
    maxeval -- the maximum number of evaluations for the LAMMPS minimization. Default value is 100000.    
    """
    
    #Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Read LAMMPS input template
    with open('min.template') as f:
        template = f.read()
    
    #Define lammps variables
    lammps_variables = {}
    lammps_variables['atomman_system_info'] = lmp.atom_data.dump(system, 'perfect.dat', 
                                                                 units=potential.units, 
                                                                 atom_style=potential.atom_style)
    lammps_variables['atomman_pair_info'] =   potential.pair_info(symbols)
    lammps_variables['energy_tolerance'] =    etol
    lammps_variables['force_tolerance'] =     uc.get_in_units(ftol, lammps_units['force'])
    lammps_variables['maximum_iterations'] =  maxiter
    lammps_variables['maximum_evaluations'] = maxeval

    #Write lammps input script
    with open('min.in', 'w') as f:
        f.write('\n'.join(iprPy.tools.fill_template(template, lammps_variables, '<', '>')))

    #run lammps to relax perfect.dat
    output = lmp.run(lammps_command, 'min.in', mpi_command)
    
    #Extract LAMMPS thermo data.
    E_total_base = uc.set_in_units(output.finds('PotEng')[-1], lammps_units['energy'])
    E_coh =        uc.set_in_units(output.finds('peatom')[-1], lammps_units['energy'])
    
    #rename log file
    shutil.move('log.lammps', 'min-perfect-log.lammps')
    
    #Load relaxed system from dump file and copy old vects as dump files crop values
    last_dump_file = 'atom.'+str(int(output.finds('Step')[-1]))
    system_base = lmp.atom_dump.load(last_dump_file)
    system_base.box_set(vects=system.box.vects)
    
    #Add defect(s)
    system_ptd = deepcopy(system_base)
    for params in ptd_model.iterfinds('atomman-defect-point-parameters'):       
        system_ptd = am.defect.point(system_ptd, **params)
    
    #update lammps variables
    lammps_variables['atomman_system_info'] = lmp.atom_data.dump(system_ptd, 'defect.dat',
                                                                 units = potential.units, 
                                                                 atom_style = potential.atom_style)
    
    #Write lammps input script
    with open('min.in', 'w') as f:
        f.write('\n'.join(iprPy.tools.fill_template(template, lammps_variables, '<', '>')))

    #run lammps
    output = lmp.run(lammps_command, 'min.in', mpi_command)
    
    #extract lammps thermo data
    E_total_ptd = uc.set_in_units(output.finds('PotEng')[-1], lammps_units['energy'])
    
    #rename log file
    shutil.move('log.lammps', 'min-defect-log.lammps')    
    
    #Load relaxed system from dump file and copy old vects as dump files crop values
    last_dump_file = 'atom.'+str(int(output.finds('Step')[-1]))
    system_ptd = lmp.atom_dump.load(last_dump_file)
    system_ptd.box_set(vects=system.box.vects)
    
    #compute defect formation energy as difference between total potential energy of defect system
    #and the cohesive energy of the perfect system times the number of atoms in the defect system
    E_ptd_f = E_total_ptd - E_coh * system_ptd.natoms
    
    return {'E_coh':       E_coh,        'E_ptd_f':    E_ptd_f, 
            'E_total_base':E_total_base, 'E_total_ptd':E_total_ptd,
            'system_base': system_base,  'system_ptd': system_ptd}

#### 3.2 check_ptd_config Function

Evaluates a relaxed system containing a point defect to determine if the defect structure has transformed to a different configuration.

Arguments:

- __system__ -- atomman.System containing the point defect(s).

- __ptd_model__ -- DataModelDict representation of a point defect data model. This identifies the defect configuration that was added to system.

- __cutoff__ -- cutoff to use for identifying atoms nearest to the defect's position.
    
Returns a dictionary containing:

- __has_reconfigured__ -- Boolean indicating if any of the algorithms reveal that the defect configuration has changed. 

- __centrosummation__ -- sums up the vector positions of atoms relative to the defect's center. Most characterized point defect configurations are centered on lattice symmetry sites, and for most of those lattice symmetry sites the centrosummation will be zero. A value not near [0,0,0] indicates that the local symmetry around the defect's position has been broken. Only neighboring atoms within cutoff are included. 

- __position_shift__ -- the vector difference between where an interstitial or substitutional atom's relaxed and initial positions. A value not near [0,0,0] indicates that the defect atom has moved. Only calculated for position-based interstitials and substitutional defects. 

- __db_vect_shift__ -- the difference between a dumbbell interstitial's normalized vector before and after relaxation. A value not near [0,0,0] indicates that the dumbbell has rotated to a different configuration. Only calculated for dumbbell defects. 

This code is directly copied from calc_point_defect_formation_static.py.

In [9]:
def check_ptd_config(system, ptd_model, cutoff, tol=1e-5):
    """
    Evaluates a relaxed system containing a point defect to determine if the defect 
    structure has transformed to a different configuration.
    
    Arguments:
    system -- atomman.System containing the point defect(s)
    ptd_model -- DataModelDict representation of a point defect data model
    cutoff -- cutoff to use for identifying atoms nearest to the defect's position
    
    Keyword Argument:
    tol -- tolerance to use for identifying if a defect has reconfigured. Default value is 1e-5.
    """
    
    #Extract the parameter sets
    params = deepcopy(ptd_model.finds('atomman-defect-point-parameters'))
    
    #if there is only one set, use that set
    if len(params) == 1:
        params = params[0]
        
    #if there are two sets (divacancy), use the first and average position
    elif len(params) == 2:
        di_pos = (np.array(params[0]['pos']) + np.array(params[1]['pos'])) / 2
        params = params[0]
        params['pos'] = di_pos
    
    #More than two not supported by this function
    else:
        raise ValueError('Invalid point defect parameters')
    
    #Initially set has_reconfigured to False
    has_reconfigured = False
    
    #Calculate distance of all atoms from defect position
    pos_vects = system.dvect(system.atoms.view['pos'], params['pos']) 
    pos_mags = np.linalg.norm(pos_vects, axis=1)
    
    #calculate centrosummation by summing up the positions of the close atoms
    centrosummation = np.sum(pos_vects[pos_mags < cutoff], axis=0)
    
    if not np.allclose(centrosummation, np.zeros(3), atol=tol):
        has_reconfigured = True
        
    #Calculate shift of defect atom's position if interstitial or substitutional
    if params['ptd_type'] == 'i' or params['ptd_type'] == 's':
        position_shift = system.dvect(system.natoms-1, params['pos'])
       
        if not np.allclose(position_shift, np.zeros(3), atol=tol):
            has_reconfigured = True
        
        return {'has_reconfigured': has_reconfigured, 
                'centrosummation':  centrosummation, 
                'position_shift':   position_shift}
        
    #Investigate if dumbbell vector has shifted direction 
    elif params['ptd_type'] == 'db':
        db_vect = params['db_vect'] / np.linalg.norm(params['db_vect'])
        new_db_vect = system.dvect(system.natoms-2, system.natoms-1)
        new_db_vect = new_db_vect / np.linalg.norm(new_db_vect)
        db_vect_shift = db_vect - new_db_vect
        
        if not np.allclose(db_vect_shift, np.zeros(3), atol=tol):
            has_reconfigured = True
    
        return {'has_reconfigured': has_reconfigured, 
                'centrosummation':  centrosummation, 
                'db_vect_shift':    db_vect_shift}   
    
    else:
        return {'has_reconfigured': has_reconfigured, 
                'centrosummation':  centrosummation}

### 4. Run Calculation and Display Results

Specify run parameters

In [10]:
#Commands for LAMMPS and MPI
lammps_command = 'lmp_serial'
mpi_command =    None

#Paths to the potential data model and directory of artifacts
potential_file = '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json' #Generated above
potential_dir  = ''                                     #Files generated above in working directory

#System load parameters
load_style = 'system_model'
load_file =  'A1--Cu--fcc.json' #Generated above

#Specify symbols combinations to use
symbols = ['AlS']

#Specify lattice parameters to use
a = b = c = uc.set_in_units(4.05, 'angstrom')

#Specify system size multipliers
a_mult = 5
b_mult = 5
c_mult = 5

#List point defect models to include
ptd_model_files = ['100-dumbbell.json', 
                   '110-dumbbell.json', 
                   '111-dumbbell.json', 
                   'octahedral-interstitial.json',
                   'tetrahedral-interstitial.json',
                   'crowdion-interstitial.json']

Read the interatomic potential data model

In [11]:
with open(potential_file) as f:
    potential = lmp.Potential(f, potential_dir)

Read the prototype unit cell system

In [12]:
ucell = am.load(load_style, load_file)[0]

Resize the unit cell according to the initial lattice parameter guess

In [13]:
ucell.box_set(a=a, b=b, c=c, scale=True)

Generate a supercell system using size multipliers

In [14]:
system = am.supersize(ucell, a_mult, b_mult, c_mult)

Run ptd_energy and check_ptd_config, and display results

In [15]:
for ptd_model_file in ptd_model_files:
    #Load ptd_model
    with open(ptd_model_file) as f:
        ptd_model = DM(f)

    #print defect name
    ptd_name = ptd_model['point-defect-parameters']['point-defect']['id']    
    print ptd_name
    
    #Unscale vector parameters relative to ucell
    for params in ptd_model.iterfinds('atomman-defect-point-parameters'):
        if 'scale' in params and params['scale'] is True:
            if 'pos' in params:
                params['pos'] = list(ucell.unscale(params['pos']))
            if 'db_vect' in params:
                params['db_vect'] = list(ucell.unscale(params['db_vect']))
            params['scale'] = False
    
    #call ptd_energy
    results = ptd_energy(lammps_command, system, potential, symbols, ptd_model)
    
    #print results
    print 'E_ptd_f =', uc.get_in_units(results['E_ptd_f'], 'eV'), 'eV',
    
    #Save final configurations to dump files    
    lmp.atom_dump.dump(results['system_ptd'], ptd_name+'.dump')
    
    #Call check_ptd_config
    config = check_ptd_config(results['system_ptd'], ptd_model, a)
    
    #print results
    if config['has_reconfigured']:
        print 'Has reconfigured!'
    else:
        print
        
    centrosummation = uc.get_in_units(config['centrosummation'], 'angstrom')
    print 'centrosummation = [%.3f, %.3f, %.3f] A' % (centrosummation[0], centrosummation[1], centrosummation[2])
    
    if 'position_shift' in config:
        position_shift = uc.get_in_units(config['position_shift'], 'angstrom')
        print 'position_shift  = [%.3f, %.3f, %.3f] A' % (position_shift[0], position_shift[1], position_shift[2])
        
    if 'db_vect_shift' in config:
        db_vect_shift = config['db_vect_shift']
        print 'db_vect_shift   = [%6.3f, %6.3f, %6.3f]' % (db_vect_shift[0], db_vect_shift[1], db_vect_shift[2])
    print

100-dumbbell
E_ptd_f = 2.79013908288 eV
centrosummation = [0.000, 0.000, 0.000] A
db_vect_shift   = [ 0.000, -0.000,  0.000]

110-dumbbell
E_ptd_f = 2.80786680178 eV
centrosummation = [-0.000, -0.000, -0.000] A
db_vect_shift   = [-0.000,  0.000, -0.000]

111-dumbbell
E_ptd_f = 2.80786680178 eV Has reconfigured!
centrosummation = [0.000, 0.000, 0.000] A
db_vect_shift   = [-0.130,  0.577, -0.130]

octahedral-interstitial
E_ptd_f = 3.28931256138 eV
centrosummation = [-0.000, -0.000, -0.000] A
position_shift  = [-0.000, -0.000, -0.000] A

tetrahedral-interstitial
E_ptd_f = 3.34092910008 eV
centrosummation = [-0.000, -0.000, 0.000] A
position_shift  = [0.000, 0.000, -0.000] A

crowdion-interstitial
E_ptd_f = 2.80786680178 eV Has reconfigured!
centrosummation = [-0.307, -0.307, -0.000] A
position_shift  = [-0.232, -0.232, -0.000] A

